# Webscraping allrecipes.com
<b>Edie Espejo</b>

<b>Link</b>: http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201

# Install BeautifulSoup

In [1]:
# import pip
# pip.main(["install", "bs4"])

In [2]:
# pip.main(["install", "requests_cache"])

In [3]:
# import pip
# pip.main(["install", "time"])

In [4]:
# import time 
# while True:
#     print("This prints once a minute.")
#     time.sleep(60)   # Delay for 1 minute (60 seconds).

In [5]:
# IMPORTANT LIBRARIES
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import requests_cache
import os

In [6]:
os.getcwd()

'C:\\Users\\ehze\\Documents'

### Don't forget to CACHE!

In [7]:
# START CACHING
requests_cache.install_cache("C://Users/ehze/Documents/banana_cache_money")

# Collect Links
It seems that the links are formatted in this fashion:
http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=5#5

where we can replace the last numbers to resemble integers from 2 to 9.

In [8]:
# FORMAT LINKS
home = "http://allrecipes.com"
base_link = home + "/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page="

all_links = [base_link+str(integer)+"#"+str(integer) for integer in range(1,10)]

In [9]:
all_links

['http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=1#1',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=2#2',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=3#3',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=4#4',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20results&clickId=cardslot%201&page=5#5',
 'http://allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/?internalSource=hubcard&referringContentType=search%20re

The markup sort of looks like this.

        <h3 class="grid-col__h3 grid-col__h3--recipe-grid">
            Chef John's Banana Bread
                <div class="grid-col__video">
                    <a href="/video/528/chef-johns-banana-bread/" data-internal-referrer-link='hub recipe' data-click-id='cardslot 1'><span class="icon--videoplay-small-white"></span></a>
                </div>
        </h3>

In [10]:
# FUNCTION TO RETRIEVE URLS FROM ONE PAGE
def one_page_recipes(url):
    soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    page_recipes = soup.find_all("article", {"class", "grid-col--fixed-tiles"})
    
    # INITIALIZE LIST
    recipes = []
    for page in page_recipes:
        try:
            temp = page.find_all("a")[0]["href"]
            if temp.startswith("/"):
                recipes.append(temp)
        except:
            continue
                  
    return(recipes)

In [11]:
banana_bread_links = [one_page_recipes(this_link) for this_link in all_links]

In [12]:
# len(banana_bread_links)

In [13]:
# CONCATENATE LIST AND FORMAT UNIVERSAL LINKS
bblinks = [home+list_item for sublist in banana_bread_links for list_item in sublist if "video" not in list_item]

In [14]:
bblinks

['http://allrecipes.com/recipe/23082/janets-famous-banana-nut-bread/',
 'http://allrecipes.com/recipe/6765/4h-banana-bread/',
 'http://allrecipes.com/recipe/218541/caramel-macchiato-banana-bread/',
 'http://allrecipes.com/recipe/219967/chef-johns-banana-bread/',
 'http://allrecipes.com/recipe/7154/creamy-banana-bread/',
 'http://allrecipes.com/recipe/17252/chocolate-banana-bread/',
 'http://allrecipes.com/recipe/228442/raspberry-banana-bread/',
 'http://allrecipes.com/recipe/16999/grandmas-homemade-banana-bread/',
 'http://allrecipes.com/recipe/141933/banana-zucchini-bread/',
 'http://allrecipes.com/recipe/17652/banana-crumb-muffins/',
 'http://allrecipes.com/recipe/20144/banana-banana-bread/',
 'http://allrecipes.com/recipe/17066/janets-rich-banana-bread/',
 'http://allrecipes.com/recipe/6984/banana-sour-cream-bread/',
 'http://allrecipes.com/recipe/67937/extreme-banana-nut-bread-ebnb/',
 'http://allrecipes.com/recipe/15747/best-ever-banana-bread/',
 'http://allrecipes.com/recipe/1695

In [15]:
len(bblinks)

177

Now the links have been collected, let's continue on to check out how to read one of the pages.

# Gathering Page Information
We're going to inspect one of the pages, then create a function that can be generalized to retrieve:
<br>
(1) Recipe Name
<br>
(2) Star Rating & Review Count
<br>
(4) Ingredients
<br>
(5) Calories

In [16]:
bblinks[0]

'http://allrecipes.com/recipe/23082/janets-famous-banana-nut-bread/'

### Recipe Name
The markup for the recipe name looks like this. 

In [17]:
# <section class="recipe-summary clearfix">
#     <meta property="og:rating" content="4.41201734542847" />
#     <meta property="og:rating_scale" content="5" />
#     <div id="karma-lazy-sponsorLogo" class="recipe-summary__logo"></div>
#     <h1 class="recipe-summary__h1" itemprop="name">Strawnana Bread</h1>
#     <!--rating stars-->
#     <div class="recipe-summary__stars">
#         <div class="rating-stars" data-scroll-to-anchor="reviews" data-ratingstars= 4.41201734542847 >

In [18]:
#         <span itemprop="aggregateRating" itemscope itemtype="http://schema.org/AggregateRating" class="aggregate-rating">
#             <meta itemprop="ratingValue" content="4.41">
#             <meta itemprop="reviewCount" content="182">

In [19]:
soup = BeautifulSoup((requests.get(bblinks[0])).text, "html.parser")
clearfix = soup.find_all("section", {"class", "recipe-summary clearfix"})[0]

In [20]:
clearfix

<section class="recipe-summary clearfix">
<meta content="4.63492059707642" property="og:rating"/>
<meta content="5" property="og:rating_scale"/>
<div class="recipe-summary__logo" id="karma-lazy-sponsorLogo"></div>
<h1 class="recipe-summary__h1" itemprop="name">Janet's Famous Banana Nut Bread</h1>
<!--rating stars-->
<div class="recipe-summary__stars">
<div class="rating-stars" data-ratingstars="4.63492059707642" data-scroll-to-anchor="reviews">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating-stars/full-star-2015.svg" width="20">
<img height="20" src="http://images.media-allrecipes.com/ar-images/icons/rating

In [21]:
clearfix.find("h1").contents[0]

"Janet's Famous Banana Nut Bread"

### Star Rating & Review Count

In [22]:
clearfix.find("div", {"class": "recipe-summary__stars"}).find("div", {"class", "rating-stars"})["data-ratingstars"]

'4.63492059707642'

In [23]:
metas = clearfix.find("div", {"class": "recipe-summary__stars"}).find("span", {"class", "aggregate-rating"}).find_all("meta")

In [24]:
metas

[<meta content="4.63" itemprop="ratingValue"/>,
 <meta content="110" itemprop="reviewCount"/>]

In [25]:
[h["content"] for h in metas]

['4.63', '110']

### Ingredients

In [28]:
ingredients = soup.find_all("span", {"class", "recipe-ingred_txt added"})

In [29]:
len(ingredients)

10

In [30]:
[item.contents[0] for item in ingredients]

['3 cups white sugar',
 '3/4 cup butter, softened',
 '3 eggs',
 '2 cups mashed ripe banana',
 '1/2 cup sour cream',
 '3 1/2 cups all-purpose flour',
 '1 teaspoon baking soda',
 '1/2 teaspoon baking powder',
 '1/2 teaspoon ground cinnamon',
 '1 1/2 cups chopped pecans']

### Number of Ratings

In [31]:
# <span class="calorie-count" ng-class="{'active': nutritionSection_showing}"><span>258</span> <span class="calorie-count__desc" ng-class="{'active': nutritionSection_showing}"> cals</span></span>

In [32]:
soup.find("h4", {"class", "helpful-header"}).contents[0]

'126 Ratings'

### Calories

In [33]:
soup.find_all("span", {"class", "calorie-count"})[0].find("span").contents[0]

'356'

### Number Made It

In [93]:
soup.find("div", {"class", "summary-stats-box"}).find("div")["data-ng-init"][4:]

'(151,null)'

### Number Reviews

In [98]:
soup.find("span", {"class", "review-count"}).contents[0]

'110 reviews'

### Number of Photos

In [ ]:
# <span>&nbsp; | &nbsp;</span><span class="picture-count-link"><format-large-number number="7"></format-large-number> photos</span>


In [96]:
# soup.find("span", {"class", "picture-count-link"})

<span class="picture-count-link"><format-large-number number="7"></format-large-number> photos</span>

### Prep Times

In [74]:
[x.contents[0] for x in soup.find("ul", {"class", "prepTime"}).find_all("li")[1].find_all("span")]

['20']

In [75]:
[x.contents[0] for x in soup.find("ul", {"class", "prepTime"}).find_all("li")[2].find_all("span")]

['1', '5']

In [76]:
[x.contents[0] for x in soup.find("ul", {"class", "prepTime"}).find_all("li")[3].find_all("span")]

['1', '25']

### Function To Retrieve All
Now, what remains is the task to make a function to wrangle these data from a generic page!

In [35]:
# def read_a_page(url):
#     # CONVERT PAGE
#     soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    
#     # RECIPE TITLE
#     try:
#         clearfix = soup.find_all("section", {"class", "recipe-summary clearfix"})[0]
#         title = clearfix.find("h1").contents[0]
#     except:
#         print(url)

#     # RATINGS
#     try:
#         longRating = clearfix.find("div", {"class": "recipe-summary__stars"}).find("div", {"class", "rating-stars"})["data-ratingstars"]
#         metas = clearfix.find("div", {"class": "recipe-summary__stars"}).find("span", {"class", "aggregate-rating"}).find_all("meta")
#         metas_content = [h["content"] for h in metas]
#         ratingValue, reviewCount = metas_content[0], metas_content[1]
#     except:
#         print(url)
    
#     # INGREDIENTS
#     try:
#         ingredients = soup.find_all("span", {"class", "recipe-ingred_txt added"})
#         recipe = [item.contents[0] for item in ingredients]
#     except:
#         print(url)
                         
#     # CALORIES
#     try:
#         calories = soup.find_all("span", {"class", "calorie-count"})[0].find("span").contents[0]
#     except:
#         print(url)
    
#     try:
#         return([title, longRating, ratingValue, reviewCount, recipe, calories])
#     except:
#         return(0)

In [99]:
def read_a_page(url):
    # CONVERT PAGE
    soup = BeautifulSoup((requests.get(url)).text, "html.parser")
    
    # RECIPE TITLE
    try:
        clearfix = soup.find_all("section", {"class", "recipe-summary clearfix"})[0]
        title = clearfix.find("h1").contents[0]
    except:
        title = np.NaN
        
    # MADE IT
    try:
        made_it = soup.find("div", {"class", "summary-stats-box"}).find("div")["data-ng-init"][4:]
    except:
        made_it = np.NaN
    
    # REVIEWS
    try:
        reviews = soup.find("span", {"class", "review-count"}).contents[0]
    except:
        reviews = np.NaN

    # RATINGS
    longRating = clearfix.find("div", {"class": "recipe-summary__stars"}).find("div", {"class", "rating-stars"})["data-ratingstars"]
    metas = clearfix.find("div", {"class": "recipe-summary__stars"}).find("span", {"class", "aggregate-rating"}).find_all("meta")
    metas_content = [h["content"] for h in metas]
    ratingValue, reviewCount = metas_content[0], metas_content[1]
    
    # INGREDIENTS
    ingredients = soup.find_all("span", {"class", "recipe-ingred_txt added"})
    recipe = [item.contents[0] for item in ingredients]
                         
    # CALORIES
    try:
        calories = soup.find_all("span", {"class", "calorie-count"})[0].find("span").contents[0]
    except:
        calories = np.NaN
    
    # NUMBER OF RATINGS
    try:
        ratings = soup.find("h4", {"class", "helpful-header"}).contents[0]
    except:
        ratings = np.NaN
    
    # PREP TIMES
    try:
        prep = [x.contents[0] for x in soup.find("ul", {"class", "prepTime"}).find_all("li")[1].find_all("span")]
        cook = [x.contents[0] for x in soup.find("ul", {"class", "prepTime"}).find_all("li")[2].find_all("span")]
        total = [x.contents[0] for x in soup.find("ul", {"class", "prepTime"}).find_all("li")[3].find_all("span")]
    except:
        prep, cook, total = np.NaN, np.NaN, np.NaN
    
    return([url, title, made_it, reviews, longRating, ratingValue, reviewCount, recipe, calories, ratings, prep, cook, total])                     
    

In [100]:
table = pd.DataFrame([read_a_page(this_link) for this_link in bblinks])

In [ ]:
# i = 0
# table_tester = []
# for this_link in bblinks:
#     print(i)
#     try:
#         table_tester.append((read_a_page(this_link)))
#     except:
#         table_tester.append([np.NaN]*8)
#     i = i + 1

In [101]:
table.columns = ["url", "title", "made-it", "reviews", "longRating", "ratingValue", "reviewCount", "recipe", "calories", "ratings", "prep", "cook", "total"]

In [102]:
table

,url,title,made-it,reviews,longRating,ratingValue,reviewCount,recipe,calories,ratings,prep,cook,total
0,http://allrecipes.com/recipe/23082/janets-famo...,Janet's Famous Banana Nut Bread,"(151,null)",110 reviews,4.63492059707642,4.63,110,"[3 cups white sugar, 3/4 cup butter, softened,...",356,126 Ratings,[20],"[1, 5]","[1, 25]"
1,http://allrecipes.com/recipe/6765/4h-banana-br...,4H Banana Bread,"(340,null)",881 reviews,4.67164182662964,4.67,881,"[2 cups all-purpose flour, 1/2 teaspoon baking...",265,1072 Ratings,[10],[1],"[1, 10]"
2,http://allrecipes.com/recipe/218541/caramel-ma...,Caramel Macchiato Banana Bread,"(188,null)",57 reviews,4.18181800842285,4.18,57,"[cooking spray, 2 cups all-purpose flour, 1 te...",334,66 Ratings,[15],"[1, 5]",[2]
3,http://allrecipes.com/recipe/219967/chef-johns...,Chef John's Banana Bread,"(981,null)",381 reviews,4.69473695755005,4.69,381,"[cooking spray, 2 cups all-purpose flour, 1 te...",334,475 Ratings,[15],"[1, 10]","[1, 45]"
4,http://allrecipes.com/recipe/7154/creamy-banan...,Creamy Banana Bread,"(342,null)",869 reviews,4.6641001701355,4.66,869,"[1/2 cup margarine, softened, 1 (8 ounce) pack...",289,1039 Ratings,[30],[45],"[1, 15]"
5,http://allrecipes.com/recipe/17252/chocolate-b...,Chocolate Banana Bread,"(1449,null)",925 reviews,4.67378330230713,4.67,925,"[1 cup margarine, softened, 2 cups white sugar...",278,1171 Ratings,[10],[1],"[1, 10]"
6,http://allrecipes.com/recipe/228442/raspberry-...,Raspberry Banana Bread,"(95,null)",32 reviews,4.68292665481567,4.68,32,"[3/4 cup frozen unsweetened raspberries, thawe...",273,41 Ratings,[25],[1],"[1, 55]"
7,http://allrecipes.com/recipe/16999/grandmas-ho...,Grandma's Homemade Banana Bread,"(586,null)",312 reviews,4.69541788101196,4.70,312,"[1 1/2 cups white sugar, 1/2 cup butter, softe...",226,371 Ratings,[10],[1],"[1, 10]"
8,http://allrecipes.com/recipe/141933/banana-zuc...,Banana-Zucchini Bread,"(519,null)",263 reviews,4.51702785491943,4.52,263,"[3 eggs, 3/4 cup vegetable oil, 2/3 cup packed...",272,323 Ratings,[15],[50],"[1, 5]"
9,http://allrecipes.com/recipe/17652/banana-crum...,Banana Crumb Muffins,"(14431,null)",9640 reviews,4.803551197052,4.80,9640,"[1 1/2 cups all-purpose flour, 1 teaspoon baki...",263,13123 Ratings,[15],[20],[35]


In [104]:
table["url"][156]

'http://allrecipes.com/recipe/219863/melted-banana-bread/'

In [105]:
# table.to_csv("banana_bread_df.csv")

In [103]:
table["calories"][50]

'232'

NUMBER 47 IS A BITCH

In [ ]:
ghetto = [int(x) for x in table["calories"][0:46]] + [int(x) for x in table["calories"][48:]]

In [ ]:
ghetto

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (18, 12)
import seaborn as sns

In [ ]:
# SCORE DISTRIBUTION HISTOGRAM
x = pd.Series(ghetto)

sns.distplot(x, color="blue")
plt.title("Distribution of Calories of Banana Bread")

sns.plt.show()

In [ ]:
np.mean(ghetto)

In [ ]:
np.std(ghetto)